In [1]:
!pip install -q transformers datasets
!pip install -q lightning
!pip install -q watermark
!pip install -q rich

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!git clone https://github.com/anilbhatt1/ERA1_S15_transformers.git

Cloning into 'ERA1_S15_transformers'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 67 (delta 36), reused 50 (delta 19), pack-reused 0
Receiving objects: 100% (67/67), 21.10 KiB | 5.28 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [17]:
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks import RichProgressBar
import torchmetrics
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
from tqdm.notebook import trange, tqdm
cuda = torch.cuda.is_available()
print(f'Pytorch Version : {torch.__version__}, cuda:{cuda}')
from datasets import load_dataset
from pathlib import Path
from watermark import watermark
print(watermark(packages="lightning"))
from torch.utils.tensorboard import SummaryWriter
from tensorboard.backend.event_processing import event_accumulator
%load_ext tensorboard

Pytorch Version : 2.0.1+cu118, cuda:True
lightning: 2.0.8

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [6]:
import os
os.chdir('/content/ERA1_S15_transformers/new_lightning_modules')
print(os.getcwd())
!ls

/content/ERA1_S15_transformers/new_lightning_modules
config.py  datamodule.py  dataset.py  litmodel.py  model.py  utils.py


In [7]:
from litmodel import LightningModel
from datamodule import OpusbooksDataModule
from utils import get_model
from config import get_config
cfg = get_config()
cfg['batch_size'] = 16
cfg['preload'] = None
cfg['num_epochs'] = 1
num_val_batches = 32

cfg

{'batch_size': 16,
 'num_epochs': 1,
 'lr': 0.0001,
 'seq_len': 350,
 'd_model': 512,
 'lang_src': 'en',
 'lang_tgt': 'it',
 'model_folder': 'weights',
 'model_basename': 'tmodel_',
 'preload': None,
 'tokenizer_file': 'tokenizer_{0}.json',
 'experiment_name': 'runs/tmodel'}

In [8]:
dm = OpusbooksDataModule(cfg)
tokenizer_src, tokenizer_tgt = dm.setup(None)
vocab_src_len = tokenizer_src.get_vocab_size()
vocab_tgt_len = tokenizer_tgt.get_vocab_size()
print(vocab_src_len, vocab_tgt_len)

Generating train split:   0%|          | 0/32332 [00:00<?, ? examples/s]

train_ds_size : 29098 & val_ds_size - 3234
Max length of source sentence: 309
Max length of target sentence: 274
15698 22463


In [9]:
pytorch_model = get_model(cfg, vocab_src_len, vocab_tgt_len)

In [ ]:
'''
class LightningModel(L.LightningModule):
    def __init__(self, model, learning_rate, tokenizer_src, tokenizer_tgt, max_len, num_examples):

lightning_model = LightningModel(pytorch_model, cfg['lr'], tokenizer_src, tokenizer_tgt, cfg['seq_len'], num_examples = 32)
'''

In [10]:
callbacks = [
    ModelCheckpoint(save_top_k=1, mode="min", monitor="train_loss", save_last=True),
    RichProgressBar(leave=True)
]

In [12]:
ckpt_path_gdrive = '/content/gdrive/MyDrive/ERA1/S15_Transformers_PL/epoch=7-step=12733.ckpt'

lightning_model = LightningModel.load_from_checkpoint(ckpt_path_gdrive, model=pytorch_model)

In [13]:
trainer = L.Trainer(
    limit_val_batches=num_val_batches,
    callbacks=callbacks,
    max_epochs=3,
    accelerator="gpu",
    devices="auto",
    default_root_dir='/content/tensorboard',
)

INFO: GPU available: True (cuda), used: True
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [14]:
trainer.fit(model=lightning_model, datamodule=dm)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


train_ds_size : 29098 & val_ds_size - 3234
Max length of source sentence: 309
Max length of target sentence: 274


┏━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name        ┃ Type             ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ model       │ Transformer      │ 75.1 M │
│ 1 │ loss_fn     │ CrossEntropyLoss │      0 │
│ 2 │ cer_metric  │ CharErrorRate    │      0 │
│ 3 │ wer_metric  │ WordErrorRate    │      0 │
│ 4 │ bleu_metric │ BLEUScore        │      0 │
└───┴─────────────┴──────────────────┴────────┘

Trainable params: 75.1 M                                                                                           
Non-trainable params: 0                                                                                            
Total params: 75.1 M                                                                                               
Total estimated model params size (MB): 300

Output()

...

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/data.py:76: UserWarning: Trying to infer the 
`batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use 
`self.log(..., batch_size=batch_size)`.
  warning_cache.warn(

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/data.py:76: UserWarning: Trying to infer the 
`batch_size` from an ambiguous collection. The batch size we found is 10. To avoid any miscalculations, use 
`self.log(..., batch_size=batch_size)`.
  warning_cache.warn(

Output()

Output()

INFO: `Trainer.fit` stopped: `max_epochs=3` reached.
INFO: `Trainer.fit` stopped: `max_epochs=3` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


In [15]:
path = trainer.checkpoint_callback.best_model_path
print(path)

/content/tensorboard/lightning_logs/version_0/checkpoints/epoch=2-step=5457.ckpt


In [ ]:
%tensorboard --logdir /content/tensorboard/lightning_logs

In [16]:
!cp '/content/tensorboard/lightning_logs/version_0/checkpoints/epoch=2-step=5457.ckpt' '/content/gdrive/MyDrive/ERA1/S15_Transformers_PL'

In [18]:
log_dir = '/content/tensorboard/lightning_logs/version_0'
event_acc = event_accumulator.EventAccumulator(log_dir)
event_acc.Reload()

In [19]:
logged_scalars = {}

for tag in event_acc.Tags()["scalars"]:
    events = event_acc.Scalars(tag)
    values = [event.value for event in events]
    steps = [event.step for event in events]
    logged_scalars[tag] = (steps, values)

In [24]:
log_keys = list(logged_scalars.keys())
log_keys

['hp_metric',
 'train_loss_step',
 'epoch',
 'val_cer',
 'val_wer',
 'val_bleu',
 'train_loss_epoch']

In [22]:
len(logged_scalars['train_loss_step'][0]), len(logged_scalars['train_loss_step'][1])

(109, 109)

In [31]:
for key in log_keys:
    print(f'Metrics - {key}')
    for i in range(len(logged_scalars[key][0])):
        if key == 'train_loss_step':
            print(f'Step - {logged_scalars[key][0][i]} loss - {logged_scalars[key][1][i]:.3f}')
        elif key == 'train_loss_epoch':
            print(f'Epoch - {i} loss - {logged_scalars[key][1][i]:.3f}')
        elif key == 'val_cer':
            print(f'Epoch - {i} CER - {logged_scalars[key][1][i]:.3f}')
        elif key == 'val_wer':
            print(f'Epoch - {i} WER - {logged_scalars[key][1][i]:.3f}')
        elif key == 'val_bleu':
            print(f'Epoch - {i} Bleu - {logged_scalars[key][1][i]:.3f}')

Metrics - hp_metric
Metrics - train_loss_step
Step - 49 loss - 4.531
Step - 99 loss - 4.403
Step - 149 loss - 4.367
Step - 199 loss - 4.261
Step - 249 loss - 4.134
Step - 299 loss - 4.338
Step - 349 loss - 4.121
Step - 399 loss - 4.355
Step - 449 loss - 4.639
Step - 499 loss - 4.349
Step - 549 loss - 4.222
Step - 599 loss - 4.498
Step - 649 loss - 4.562
Step - 699 loss - 4.601
Step - 749 loss - 4.279
Step - 799 loss - 3.734
Step - 849 loss - 4.657
Step - 899 loss - 4.045
Step - 949 loss - 4.386
Step - 999 loss - 4.315
Step - 1049 loss - 4.216
Step - 1099 loss - 4.297
Step - 1149 loss - 4.265
Step - 1199 loss - 4.704
Step - 1249 loss - 4.379
Step - 1299 loss - 4.192
Step - 1349 loss - 4.728
Step - 1399 loss - 4.465
Step - 1449 loss - 4.239
Step - 1499 loss - 4.281
Step - 1549 loss - 4.199
Step - 1599 loss - 4.581
Step - 1649 loss - 4.536
Step - 1699 loss - 4.290
Step - 1749 loss - 4.472
Step - 1799 loss - 4.248
Step - 1849 loss - 4.192
Step - 1899 loss - 3.876
Step - 1949 loss - 4.117
S

In [32]:
!cp '/content/tensorboard/lightning_logs/version_0/events.out.tfevents.1693587684.da442f1950fc.1797.0' '/content/gdrive/MyDrive/ERA1/S15_Transformers_PL'

In [33]:
!cp '/content/tensorboard/lightning_logs/version_0/hparams.yaml' '/content/gdrive/MyDrive/ERA1/S15_Transformers_PL'